### DCGAN

_Implementing DC GAN for larger images_

In [1]:
#!pip install nibabel
import nibabel as nib
import numpy as np
import glob
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

2022-05-04 19:55:45.971352: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-04 19:55:45.971394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (udc-aw34-21c0): /proc/driver/nvidia/version does not exist


No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


NameError: name 'IS_COLAB' is not defined

In [ ]:
df = pd.read_csv('/project/ds6050-soa2wg/team_lambda_II/ASD_DSM_CasesvsControls.csv', sep = ',')

In [ ]:
# obtain paths for all images
images_paths_f_case = list(df.query("SEX_ == 'Female' & DX_Control == 'Autism'")['PATH'])
images_paths_f_control = list(df.query("SEX_ == 'Female' & DX_Control == 'Control'")['PATH'])
images_paths_m_case = list(df.query("SEX_ == 'Male' & DX_Control == 'Autism'")['PATH'])
images_paths_m_control = list(df.query("SEX_ == 'Male' & DX_Control == 'Control'")['PATH'])

total_cases = len(images_paths_f_case) + len(images_paths_m_case)
total_controls = len(images_paths_f_control ) + len(images_paths_m_control)
# print out number of participants per category
print('There are {} female cases'.format(str(len(images_paths_f_case))))
print('---------------------------------------')
print('There are {} female controls'.format(str(len(images_paths_f_control))))
print('---------------------------------------')
print('There are {} male cases'.format(str(len(images_paths_m_case))))
print('---------------------------------------')
print('There are {} male controls'.format(str(len(images_paths_m_control))))
print('---------------------------------------')
print(f'There are {total_cases} total cases and {total_controls} total controls! {total_controls + total_cases} participants in total')
# make one giant list
#images_paths = images_paths_f_case  + images_paths_f_control + images_paths_m_case + images_paths_m_control

# positive cases only
#images_paths = images_paths_f_case + images_paths_m_case

# female only (testing purposes)
images_paths = images_paths_f_case

# male only (testing purposes)
images_paths = images_paths_m_case

num_im = len(images_paths)
image_array = []

In [ ]:
final_list = [] # create image array from paths
for path in images_paths:
    try: 
        img = nib.load(path)
        image_data = img.get_fdata()
        image_array.append(image_data)
        final_list.append(path)
    except:
        continue
    
image_array = np.asarray(image_array).astype('float32')

In [ ]:
image_array.shape

In [ ]:
# Visualize 5 of the 391 images
for i in image_array[:5]: 
    plt.imshow(i[:,:,30])
    plt.show()

In [ ]:
# #??? Ask Chelsea about this line of code. How is this phenotype data? 
# pheno = df[['FILE_ID', 'DX_GROUP']]
# pheno_array = np.array(pheno['DX_GROUP'])
# pheno_array = np.where(pheno_array == 2, 0, pheno_array) # This is a function to replace 2 with 0
# # distribution
# df['DX_GROUP'].value_counts()

# I think she means for this to be y label data right?
y = df[['FILE_ID', 'DSM_IV_TR', 'SEX_']] # 1 means Autism, 0 means Control
#y_array = np.array(y['DSM_IV_TR'])

#positive only 
#y_array = np.array(y.query("DSM_IV_TR == 1")["DSM_IV_TR"])

# female only (testing purposes)
y_array = np.array(y.query("DSM_IV_TR == 1 & SEX_ == 'Female'")["DSM_IV_TR"])

# male only (testing purposes)
y_array = np.array(y.query("DSM_IV_TR == 1 & SEX_ == 'Male'")["DSM_IV_TR"])

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(image_array, pheno_array, test_size = 0.25, random_state = 44)
x_train, x_test, y_train, y_test = train_test_split(image_array, y_array, test_size = 0.25, random_state = 44)

In [ ]:
x_train.shape

In [ ]:
# Checking imbalance of training set. Below is how you do value counts for np array. 
(unique, counts) = np.unique(y_train, return_counts=True)
np.asarray((unique, counts)).T

In [ ]:
# first attempt at plotting 
#plt.imshow(x_train[:,:,30])
#TypeError: Invalid shape (293, 61, 61) for image data

#borrowing my other code for plotting
for i in x_train[:1]: 
    plt.imshow(i[30,:,:])
    plt.show()
    
# this code also works
plt.imshow(x_train[0][30])
plt.show()

In [ ]:
# def train_preprocessing(volume, label):
#     """Process training data by rotating and adding a channel."""
#     # Rotate volume
#     volume = rotate(volume)
#     volume = tf.expand_dims(volume, axis=3) #??? Is the extra channel by how much the image was rotated? Ask Dylan.
#     return volume, label

# def validation_preprocessing(volume, label):
#     """Process validation data by only adding a channel."""
#     volume = tf.expand_dims(volume, axis=3)
#     return volume, label

def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1 #return the integer of the division 
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image[30,:,:])
        #plt.imshow(image, cmap="binary")
        plt.axis("off")

In [ ]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train)) # This is cool, double-check the documentation for this??? 
validation_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test)) #??? Should we have equal number of each class in our training data? 

batch_size = 16
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train)) # ??? shuffle dubplicates right? Or does it only shuffle? Double-check
#     .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE) # ??? I thought prefetch was for speed purposes. What is this tf.data.AUTOTUNE. Double-check. 
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_test))
#    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# DCGAN

In [ ]:
np.random.seed(42)

codings_size = 100

generator = keras.models.Sequential([
    keras.layers.Dense(61 * 73 * 61, input_shape=[codings_size]),
    keras.layers.Reshape([61, 73, 61]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv3DTranspose(64, kernel_size=5, strides=2, padding="SAME",
                                 activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv3DTranspose(1, kernel_size=5, strides=2, padding="SAME",
                                 activation="tanh"),
])
discriminator = keras.models.Sequential([
    keras.layers.Conv3D(64, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2),
                        input_shape=[codings_size]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv3D(128, kernel_size=5, strides=2, padding="SAME",
                        activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])
gan = keras.models.Sequential([generator, discriminator])

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
#X_train_dcgan = x_train.reshape(-1, 28, 28, 1) * 2. - 1. # reshape and rescale
X_train_dcgan = x_train.reshape(-1, 61, 73, 61, 1) * 2. - 1. # reshape and rescale

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
dataset = dataset.shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
#@tf.function
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs)) # not shown in the book
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            #print(noise)
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            #print(X_fake_and_real)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            #print(y1)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1) #??? How does train_on_batch differ from fit? 
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        plot_multiple_images(generated_images, 8) # not shown 
        #plot_multiple_images(generated_images, 4) # not shown 
        plt.show() # not shown
    return generated_images # Not shown. Trying to figure out where to put this 

In [ ]:
train_gan(gan, dataset, batch_size, codings_size)